In [134]:
import nltk
import time
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
from transformers import pipeline
import torch
from nltk.tokenize import word_tokenize
from sklearn.metrics import mean_squared_error, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
from sklearn.svm import SVR, SVC
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from sklearn.decomposition import PCA
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

print('Job done-')

Job done-


In [135]:
import nltk
import time
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from sklearn.svm import SVR
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
import string
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import ElasticNet

from sklearn.preprocessing import OneHotEncoder

#### 1. 데이터 준비

In [153]:
total_file_path = "data/dataset.txt"
total_documents = []
total_document = []

with open(total_file_path, 'r', encoding='utf-8') as file:
    # Postmortum 읽기
    for line in file:
        # Document separation
        if line == '\n':
            total_documents.append('\n'.join(total_document))
            total_document = []

        # Add to document
        total_document.append(line)

    # For last game document
    total_documents.append('\n'.join(total_document))

total_documents = np.array(total_documents).reshape(-1, 1)
    
# 목표값 붙이기
file_path = 'data/classification.txt'
class_list = []

class_size = 28
with open(file_path, 'r') as file:
    for line in file:
        class_type = str(line.strip())
        class_list.append(class_type)
        
ohe = OneHotEncoder(sparse=False)
le = LabelEncoder()
le_label = le.fit_transform(np.array(class_list)).reshape(-1, 1)
ohe_label = ohe.fit_transform(np.array(class_list).reshape(-1, 1))

label_size = le_label.shape[1] + ohe_label.shape[1]
data_index = np.arange(1, total_documents.shape[0] + 1).reshape(-1, 1)
data = np.hstack((data_index, total_documents, le_label, ohe_label))

print('jobs finished b')

# 데이터셋 분할
test_ratio = 0.2
index_size = 1
X_train, X_test, y_train, y_test = train_test_split(data[:, :-label_size],
                                                    data[:, -label_size:],
                                                    test_size=test_ratio,
                                                    stratify=data[:, -label_size:-label_size + le_label.shape[1]],
                                                    random_state=0)

# 자료형 변환
X_train_index, X_test_index = X_train[:, :index_size].astype(int), X_test[:, :index_size].astype(int)
X_train, X_test = X_train[:, index_size:], X_test[:, index_size:]
X_train = X_train.flatten()
X_test = X_test.flatten()
y_train_ohe = y_train[:, le_label.shape[1]:].astype(np.float64)
y_test_ohe = y_test[:, le_label.shape[1]:].astype(np.float64)
y_train_le = y_train[:, :le_label.shape[1]].astype(np.float64)
y_test_le = y_test[:, :le_label.shape[1]].astype(np.float64)

# Result Preparation
result_table = pd.DataFrame(columns=['index', 'type', 'model', 'truth', 'prediction', 'f1', 'auc'])

jobs finished b


/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [221]:
print(total_documents.shape)
print(data_index.shape)

(668, 1)
(668, 1)


#### 2. 워드 임베딩

In [154]:
# TF-IDF 학습
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(X_train).toarray()
print(f'{X_train.shape=}')

# 검증 데이터 워드 임베딩
X_test = tfidf_vectorizer.transform(X_test).toarray()
print(f'{X_test.shape=}')

X_train.shape=(534, 4513)
X_test.shape=(134, 4513)


#### 3. 차원 축소

In [155]:
pca_var_ratio = 0.95
pca = PCA(n_components=pca_var_ratio)
original_dimension = X_train.shape[1]
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

print(f'Reduced Dimension: {original_dimension} -> {X_train.shape[1]}')

Reduced Dimension: 4513 -> 440


#### 4. Model Train

##### 4-1. LR

In [219]:
# LR 학습
timer = time.time()
lr_model = LogisticRegression(multi_class='multinomial')
lr_model.fit(X_train, y_train_le) #label-encoding 형태로 받음
print(f'Elapsed time for LR training: {(time.time() - timer)/60:0.2f} min')

# LR 학습데이터 추론
print('[LR 학습 데이터]')
train_prediction = lr_model.predict(X_train)
print(f'{y_train=}')
print(f'{train_prediction=}')

# label-encoding -> one-hot-encoding (결과 도출을 위한 형태)
train_prediction_ohe = ohe.transform(le.inverse_transform(train_prediction.astype(int)).reshape(-1, 1))

f1 = f1_score(y_train_ohe, train_prediction_ohe, average='weighted')
auc = roc_auc_score(y_train_ohe, train_prediction_ohe)
for index in range(train_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_train_index[index, 0], 'train', 'LR', y_train_le[index, 0], train_prediction[index],
                                               f1_score(y_train_ohe, train_prediction_ohe, average='weighted'),
                                               roc_auc_score(y_train_ohe, train_prediction_ohe)]
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

# LR 추론
print('[LR 검증 데이터]')
test_prediction = lr_model.predict(X_test)
print(f'{y_test=}')
print(f'{test_prediction=}')

test_prediction_ohe = ohe.transform(le.inverse_transform(test_prediction.astype(int)).reshape(-1, 1))
f1 = f1_score(y_test_ohe, test_prediction_ohe, average='weighted')
auc = roc_auc_score(y_test_ohe, test_prediction_ohe)
for index in range(test_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_test_index[index, 0], 'test', 'LR', y_test_le[index, 0], test_prediction[index],
                                               f1_score(y_test_ohe, test_prediction_ohe, average='weighted'),
                                               roc_auc_score(y_test_ohe, test_prediction_ohe)]
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Elapsed time for LR training: 0.00 min
[LR 학습 데이터]
y_train=array([['3', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
       ['5', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
       ['10', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
       ...,
       ['12', '0.0', '0.0', ..., '0.0', '1.0', '0.0'],
       ['13', '0.0', '0.0', ..., '0.0', '0.0', '1.0'],
       ['3', '0.0', '0.0', ..., '0.0', '0.0', '0.0']], dtype='<U1693')
train_prediction=array([ 3.,  9., 13., 13., 13., 13., 13., 13., 13., 13., 13., 13., 13.,
       12., 13., 13., 13., 13.,  3., 13., 13., 13., 13., 13., 13., 13.,
        9.,  9., 13., 13., 13., 13., 13., 13.,  3., 13., 13.,  3.,  3.,
        9., 13., 13.,  3.,  3., 13., 13., 13.,  9., 13.,  9., 13., 13.,
       13., 13., 13., 13., 13., 13.,  3., 13., 13., 13., 13., 13., 13.,
       13., 13.,  3., 13.,  3., 13., 13., 13., 13., 13., 13., 13.,  3.,
       13., 13., 13., 13., 13.,  3., 13., 13., 13.,  9., 13., 13., 13.,
       13., 13., 13., 13., 13.,  3., 13.,  3., 13., 13., 13., 13

##### 4-2. SVC

In [217]:
param_space = {
    'C': (1, 10000),
    'gamma': (0.01, 1.0)
}

# 하이퍼 파라미터 튜닝
svc_rbf = SVC(kernel='rbf')
opt = BayesSearchCV(svc_rbf, param_space, n_iter=50, cv=10, n_jobs=-1, n_points=8, verbose=0)

timer = time.time()


opt.fit(X_train, y_train_le.ravel())
print(f'Elapsed time for SVC optimization: {(time.time() - timer)/60:0.2f} min')
print("최적의 하이퍼파라미터:", opt.best_params_)

# SVC 학습
timer = time.time()
svc_rbf = SVC(kernel='rbf', **opt.best_params_)
svc_rbf.fit(X_train, y_train_le.ravel())
print(f'Elapsed time for SVC training: {(time.time() - timer)/60:0.2f} min')

# SVM 학습데이터 추론
print('[SVM 학습 데이터]')
train_prediction = svc_rbf.predict(X_train)
# print(f'{y_train_le=}')
# print(f'{train_prediction=}')

# label-encoding -> one-hot-encoding (결과 도출을 위한 형태)
train_prediction_ohe = ohe.transform(le.inverse_transform(train_prediction.astype(int)).reshape(-1, 1))
f1 = f1_score(y_train_ohe, train_prediction_ohe, average='weighted')
auc = roc_auc_score(y_train_ohe, train_prediction_ohe) ####################one_hot_encoding 제외
for index in range(train_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_train_index[index], 'train', 'SVM', y_train_le[index], train_prediction[index],
                                               f1_score(y_train_ohe, train_prediction_ohe, average='weighted'),
                                               roc_auc_score(y_train_ohe, train_prediction_ohe)]  ######one_hot_encoding 제외
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

# SVM 추론
print('[SVM 검증 데이터]')
test_prediction = svc_rbf.predict(X_test)
# print(f'{y_test=}')
# print(f'{test_prediction=}')
test_prediction_ohe = ohe.transform(le.inverse_transform(test_prediction.astype(int)).reshape(-1, 1))
f1 = f1_score(y_test_ohe, test_prediction_ohe, average='weighted')
auc = roc_auc_score(y_test_ohe, test_prediction_ohe) ######one_hot_encoding 제외
for index in range(test_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_test_index[index], 'test', 'SVM', y_test_le[index], test_prediction[index],
                                               f1_score(y_test_ohe, test_prediction_ohe, average='weighted'),
                                               roc_auc_score(y_test_ohe, test_prediction_ohe)] ######one_hot_encoding 제외
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(
/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(
/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(
/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(
/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  

Elapsed time for SVC optimization: 0.23 min
최적의 하이퍼파라미터: OrderedDict([('C', 7214), ('gamma', 0.08356570281256696)])
Elapsed time for SVC training: 0.00 min
[SVM 학습 데이터]
result: f1=1.000, auc=1.000
[SVM 검증 데이터]
result: f1=0.489, auc=0.598


##### 4-3. rf

In [207]:
param_space = {
    'n_estimators': Integer(10, 200),
    'max_depth': Integer(1, 32),
    'min_samples_split': Real(0.1, 1.0),
    'min_samples_leaf': Real(0.1, 0.5),
    'max_features': Categorical(['sqrt', 'log2'])
}

# 하이퍼 파라미터 튜닝
rf_model = RandomForestRegressor()
opt = BayesSearchCV(rf_model, param_space, n_iter=50, cv=10, n_jobs=-1, n_points=8, verbose=0)

timer = time.time()
opt.fit(X_train, y_train_le.ravel())
print(f'Elapsed time for RF optimization: {(time.time() - timer)/60:0.2f} min')
print("최적의 하이퍼파라미터:", opt.best_params_)

# RF 학습
timer = time.time()
rf_model = RandomForestRegressor(**opt.best_params_)
rf_model.fit(X_train, y_train_le.ravel())
print(f'Elapsed time for RF training: {(time.time() - timer)/60:0.2f} min')

# label-encoding -> one-hot-encoding (결과 도출을 위한 형태)
print('[RF 학습 데이터]')
train_prediction = rf_model.predict(X_train)
print(f'{y_train=}')
print(f'{train_prediction=}')

train_prediction_ohe = ohe.transform(le.inverse_transform(train_prediction.astype(int)).reshape(-1, 1))
f1 = f1_score(y_train_ohe, train_prediction_ohe, average='weighted')
auc = roc_auc_score(y_train_ohe, train_prediction_ohe)
for index in range(train_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_train_index[index, 0], 'train', 'RF', y_train_le[index, 0], train_prediction[index],
                                               f1_score(y_train_ohe, train_prediction_ohe, average='weighted'),
                                               roc_auc_score(y_train_ohe, train_prediction_ohe)]
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

# RF 추론
print('[RF 검증 데이터]')
test_prediction = rf_model.predict(X_test)
print(f'{y_test=}')
print(f'{test_prediction=}')

test_prediction_ohe = ohe.transform(le.inverse_transform(test_prediction.astype(int)).reshape(-1, 1))
f1 = f1_score(y_test_ohe, test_prediction_ohe, average='weighted')
auc = roc_auc_score(y_test_ohe, test_prediction_ohe)
for index in range(test_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_test_index[index, 0], 'test', 'RF', y_test_le[index, 0], test_prediction[index],
                                               f1_score(y_test_ohe, test_prediction_ohe, average='weighted'),
                                               roc_auc_score(y_test_ohe, test_prediction_ohe)]
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

Elapsed time for RF optimization: 0.30 min
최적의 하이퍼파라미터: OrderedDict([('max_depth', 14), ('max_features', 'sqrt'), ('min_samples_leaf', 0.1), ('min_samples_split', 0.15802613542759672), ('n_estimators', 143)])
Elapsed time for RF training: 0.00 min
[LR 학습 데이터]
y_train=array([['3', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
       ['5', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
       ['10', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
       ...,
       ['12', '0.0', '0.0', ..., '0.0', '1.0', '0.0'],
       ['13', '0.0', '0.0', ..., '0.0', '0.0', '1.0'],
       ['3', '0.0', '0.0', ..., '0.0', '0.0', '0.0']], dtype='<U1693')
train_prediction=array([ 8.10833497,  8.76669215,  8.82707651,  8.68755975,  8.77906401,
        8.29089203,  8.68317342,  8.34202589,  8.83664362,  9.2117771 ,
        8.55722841,  8.71360557, 11.1334706 ,  8.85508424,  8.9464327 ,
        9.28151791,  8.88628667,  8.78863945,  8.16887724, 10.93303619,
        9.30619335, 10.74661728,  8.5432492 ,  9.96183561,  9.55866551,

##### 4-4. xgb

In [209]:
param_space = {
    'learning_rate': (0.01, 1.0, 'log-uniform'),
    'max_depth': (1, 50),
    'n_estimators': (50, 200),
    'min_child_weight': (1, 10),
    'subsample': (0.1, 1.0, 'uniform'),
    'gamma': (0.1, 5.0),
    'colsample_bytree': (0.1, 1.0, 'uniform'),
    'alpha': (0.0, 1.0, 'uniform'),
    'reg_lambda': (0.0, 1.0, 'uniform')
}

# 하이퍼 파라미터 튜닝
xgb_model = xgb.XGBClassifier(objective='multi:softmax')
opt = BayesSearchCV(xgb_model, param_space, n_iter=50, cv=10, n_jobs=-1, n_points=8, verbose=0)

timer = time.time()
opt.fit(X_train, y_train_le)
print(f'Elapsed time for xgb optimization: {(time.time() - timer)/60:0.2f} min')
print("최적의 하이퍼파라미터:", opt.best_params_)

# XGB 학습
timer = time.time()
xgb_model = xgb.XGBClassifier(objective='multi:softmax', **opt.best_params_)
xgb_model.fit(X_train, y_train_le)
print(f'Elapsed time for xgb training: {(time.time() - timer)/60:0.2f} min')

# label-encoding -> one-hot-encoding (결과 도출을 위한 형태)
print('[XGB 학습 데이터]')
train_prediction = xgb_model.predict(X_train)
print(f'{y_train=}')
print(f'{train_prediction=}')

train_prediction_ohe = ohe.transform(le.inverse_transform(train_prediction.astype(int)).reshape(-1, 1))
f1 = f1_score(y_train_ohe, train_prediction_ohe, average='weighted')
auc = roc_auc_score(y_train_ohe, train_prediction_ohe)
for index in range(train_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_train_index[index, 0], 'train', 'XGB', y_train_le[index, 0], train_prediction[index],
                                               f1_score(y_train_ohe, train_prediction_ohe, average='weighted'),
                                               roc_auc_score(y_train_ohe, train_prediction_ohe)]
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

# XGB 추론
print('[XGB 검증 데이터]')
test_prediction = xgb_model.predict(X_test)
print(f'{y_test=}')
print(f'{test_prediction=}')

test_prediction_ohe = ohe.transform(le.inverse_transform(test_prediction.astype(int)).reshape(-1, 1))
f1 = f1_score(y_test_ohe, test_prediction_ohe, average='weighted')
auc = roc_auc_score(y_test_ohe, test_prediction_ohe)
for index in range(test_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_test_index[index, 0], 'test', 'XGB', y_test_le[index, 0], test_prediction[index],
                                               f1_score(y_test_ohe, test_prediction_ohe, average='weighted'),
                                               roc_auc_score(y_test_ohe, test_prediction_ohe)]
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(
/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(
/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(
/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(
/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  

Elapsed time for xgb optimization: 5.06 min
최적의 하이퍼파라미터: OrderedDict([('alpha', 1.0), ('colsample_bytree', 0.8762597116988676), ('gamma', 0.1), ('learning_rate', 0.17707746629684062), ('max_depth', 1), ('min_child_weight', 10), ('n_estimators', 200), ('reg_lambda', 1.0), ('subsample', 0.8734165115984202)])
Elapsed time for xgb training: 0.03 min
[XGB 학습 데이터]
y_train=array([['3', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
       ['5', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
       ['10', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
       ...,
       ['12', '0.0', '0.0', ..., '0.0', '1.0', '0.0'],
       ['13', '0.0', '0.0', ..., '0.0', '0.0', '1.0'],
       ['3', '0.0', '0.0', ..., '0.0', '0.0', '0.0']], dtype='<U1693')
train_prediction=array([ 3,  5, 10,  3, 13, 13,  7,  2, 13, 13, 13, 13, 13, 12, 12, 13, 13,
        8,  3, 13, 13, 13,  3, 13,  9, 13, 13,  9, 13,  8,  8,  9, 13, 13,
        3, 13, 13,  7,  3,  9, 13, 13,  3,  3,  8, 13, 13,  9,  2,  8, 13,
       12, 13,  9, 12, 13,  7, 13

In [218]:
result_table.to_excel('result/result_table_tfidf.xlsx', index=False)

pd.pivot_table(result_table,
               values=['f1', 'auc'],
               index=['type', 'model'],
               aggfunc=['mean', 'std'])

mean                 std          
                  auc        f1       auc        f1
type  model                                        
test  LR     0.534878  0.409051  0.000000  0.000000
      RF     0.505215  0.189497  0.007384  0.112854
      SVM    0.598213  0.488817  0.000000  0.000000
      XGB    0.528350  0.392949  0.000000  0.000000
train LR     0.564713  0.517675  0.000000  0.000000
      RF     0.833867  0.672275  0.235020  0.463618
      SVM    1.000000  1.000000  0.000000  0.000000
      XGB    0.830338  0.912614  0.000000  0.000000